**Ejercicio 1:** La biblioteca dispone de la función train test split, que permite generar una partición de entrenamiento y prueba, y KFold, que genera varias particiones de validación cruzada.
Estudie el uso de estas funciones y utilícelas para generar particiones a partir del conjunto de datos Digits que en Scikit-learn puede cargarse usando la función load digits del módulo datasets.
Mediante MLPClassifier construya un perceptrón multicapa adecuado para este conjunto de datos y analice su desempeño considerando:
- 1 única partición de datos genererada mediante train test split,
- 5 particiones generadas mediante KFold,
- 10 particiones generadas mediante KFold.

Para el análisis tenga en cuenta la media y la varianza de la tasa de acierto obtenida en cada partición.

#### **Librerías**

In [81]:
import numpy as np       
from tabulate import tabulate       # Para generar tablas

from sklearn import datasets                            # Módulo para cargar datos
from sklearn.neural_network import MLPClassifier        # Módulo para construir perceptrón multicapa

# Modelos de partición:
from sklearn.model_selection import train_test_split    # Caso 1
from sklearn.model_selection import KFold               # Casos 2 y 3

# Medidas de desempeño:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

#### **Inicialización**

Levanto los datos del conjunto Digits y armo mi perceptrón multicapa para probar el desempeño con las distintas particiones.

In [82]:
# Cargo los datos Digits:
X_digits,y_digits = datasets.load_digits(return_X_y=True)  

# Genero mi perceptrón multicapa:
clf = MLPClassifier(hidden_layer_sizes=(20,10),learning_rate_init=0.005, max_iter=1000, activation='logistic')

Para los casos 2 y 3, podemos generar una función auxiliar que genere n particiones según un argumento que le pasemos junto al conjunto de datos. Como KFold toma como número de particiones por defecto al 5, lo seteamos de esta manera en los argumentos de la función también.

También hacemos una función auxiliar para generar los resultados de las medidas de precisión y la tabulación por particiones.

In [83]:
def generar_kfold(X_digits,y_digits,clf,n_particiones=5):
    kf = KFold(n_splits=n_particiones)
    ACC = []

    for train_index, test_index in kf.split(X_digits):
        X_train, X_test = X_digits[train_index], X_digits[test_index]
        y_train, y_test = y_digits[train_index], y_digits[test_index]

        clf.fit(X_train, y_train)       # Entreno perceptrón con el conjunto de datos obtenido.
        y_pred = clf.predict(X_test)    # Obtengo salida con datos de prueba
        ACC_aux = accuracy_score(y_test,y_pred)
        ACC.append(ACC_aux)

    return ACC

def medidas(ACC):
    # Medidas globales
    print('Exactitud media:',np.mean(ACC))
    print('Varianza de la exactitud:',np.var(ACC))
    # Medidas tabla
    table_data = [[x,y] for x, y in zip(range(len(ACC)), ACC)]
    headers = ['N° Partición','Precisión']
    table = tabulate(table_data, headers, tablefmt='simple_grid',stralign='center',numalign='center')
    print(table)

##### **Caso 1:** Una única partición de datos generada mediante train test split.

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits, test_size=0.33, random_state=42)
clf.fit(X_train, y_train)       # Entreno perceptrón con el conjunto de datos obtenido.

y_pred = clf.predict(X_test)
ACC = accuracy_score(y_test,y_pred)
print('Exactitud:',ACC)


Exactitud: 0.9444444444444444


##### **Caso 2:** 5 particiones generadas mediante KFold

In [85]:
ACC = generar_kfold(X_digits,y_digits,clf)
medidas(ACC)


Exactitud media: 0.9198700092850511
Varianza de la exactitud: 0.0006639685369631348
┌────────────────┬─────────────┐
│  N° Partición  │  Precisión  │
├────────────────┼─────────────┤
│       0        │  0.936111   │
├────────────────┼─────────────┤
│       1        │  0.897222   │
├────────────────┼─────────────┤
│       2        │  0.913649   │
├────────────────┼─────────────┤
│       3        │  0.961003   │
├────────────────┼─────────────┤
│       4        │  0.891365   │
└────────────────┴─────────────┘


##### **Caso 3:** 10 particiones generadas mediante KFold

In [86]:
n_particiones = 10
ACC = generar_kfold(X_digits,y_digits,clf,n_particiones)
medidas(ACC)

Exactitud media: 0.9320918684047175
Varianza de la exactitud: 0.0007517147204526926
┌────────────────┬─────────────┐
│  N° Partición  │  Precisión  │
├────────────────┼─────────────┤
│       0        │  0.888889   │
├────────────────┼─────────────┤
│       1        │  0.961111   │
├────────────────┼─────────────┤
│       2        │    0.95     │
├────────────────┼─────────────┤
│       3        │  0.933333   │
├────────────────┼─────────────┤
│       4        │  0.916667   │
├────────────────┼─────────────┤
│       5        │  0.955556   │
├────────────────┼─────────────┤
│       6        │    0.95     │
├────────────────┼─────────────┤
│       7        │   0.96648   │
├────────────────┼─────────────┤
│       8        │  0.893855   │
├────────────────┼─────────────┤
│       9        │  0.905028   │
└────────────────┴─────────────┘
